<a href="https://colab.research.google.com/github/Jlyue/cats/blob/main/colab%E7%8B%97%E8%B2%93%E8%BE%A8%E8%AD%98%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

我們試著用比較多的照片來做遷移式學習, 看是否有較好的效果。這其實也是一種示範, 看我們如果收集到了一些照片, 怎麼樣整理就能做成訓練資料。

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. 讀入函式及貓狗資料

這裡是由 Google 整理的[狗貓辨識資料](https://colab.research.google.com/github/google/eng-edu/blob/master/ml/pc/exercises/image_classification_part1.ipynb#scrollTo=MLZKVtE0dSfk)讀入, 我們只會用原本準備好的測試資料, 貓狗各 500 張照片。

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

從網路讀入一個 `.zip` 檔, 存到我們 Colab 開給我們的 `/content` 資料夾下。

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /content/cats_and_dogs_filtered.zip

--2023-05-13 08:00:24--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.128, 64.233.170.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/content/cats_and_dogs_filtered.zip’

/content/cats_and_d 100%[===================>]  65.43M  21.0MB/s    in 3.7s    

2023-05-13 08:00:28 (17.9 MB/s) - ‘/content/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



這裡示範 `.zip` 檔解壓縮, 解壓縮一樣放到我們的 `/content` 資料夾中。

In [ ]:
import os
import zipfile

local_zip = '/content/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

這時在 `/content/cats_and_dogs_filtered/validation/cats` 下有 500 張貓照片, 所以我們把這個路徑記下來叫 `cats_dir`。同理我們也把狗照片路徑記下來, 叫 `dogs_dir`。

In [ ]:
base_dir = '/content/cats_and_dogs_filtered/validation'

In [ ]:
cats_dir = base_dir + '/cats'
dogs_dir = base_dir + '/dogs'

貓狗照片的檔案名稱, 分別放入 `cat_fnames` 和 `dog_fnames` 兩個串列中。

In [ ]:
cat_fnames = os.listdir(cats_dir)
dog_fnames = os.listdir(dogs_dir)

### 2. 準備好輸入輸出

現在 `data` 會放入我們轉成 `array` 的照片, 而 `target` 會是答案: 0 是貓, 狗是 1。

In [ ]:
data = []
target = []

for cat in cat_fnames:
    img = load_img(cats_dir + '/' + cat, target_size = (224,224)) #(224, 224, 3)為numpy規定img大小
    x = np.array(img)
    data.append(x)
    target.append(0)
for dog in dog_fnames:
    img = load_img(dogs_dir + '/' + dog, target_size = (224,224))
    x = np.array(img)
    data.append(x)
    target.append(1) 

In [ ]:
data = np.array(data)
target = np.array(target)

看看 `data` 的 `shape`, 會發現有 1,000 張 224x224x3 的照片。

In [ ]:
data.shape

(1000, 224, 224, 3)

`target` 自然是有 1,000 個正確答案 (貓或狗)。

In [ ]:
target.shape

(1000,)

最後我們進行 ResNet50 的標準預處理動作。

In [ ]:
data = preprocess_input(data)

### 3. 切訓練和測試資料

使用 `scikit-learn` 最常被用到的指令 `train_test_split` 切訓練和測試資料。

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, target,
                                                    test_size=0.2,
                                                    random_state=0)

### 4. Step 01: 打造我們的函數學習機

我們讀入 `ResNet50V2`, 並且去掉後面, 然後做 Global Average Pooling。注意我們讀進來 `resnet` 的權重要凍結。

最後因為我們只有兩類, 所以輸出就是一個數字! 為了確保輸出在 0 到 1 中間, 用 sigmoid 函數當我們的 activation function。

In [ ]:
resnet = ResNet50V2(include_top=False, pooling="avg")

94668760/94668760 [==============================] - 5s 0us/step


In [ ]:
resnet.trainable = False

In [ ]:
model = Sequential()

In [ ]:
model.add(resnet)

In [ ]:
model.add(Dense(1, activation='sigmoid'))

我們只有兩個類別, 所以用 `binary_crossentropy`。

In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

欣賞一下我們的成果。

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_________________________________________________________________


### 5. Step 02: 訓練

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=5)

Epoch 1/5
7/7 [==============================] - 18s 474ms/step - loss: 0.4887 - accuracy: 0.7750
Epoch 2/5
7/7 [==============================] - 2s 266ms/step - loss: 0.2126 - accuracy: 0.9375
Epoch 3/5
7/7 [==============================] - 2s 266ms/step - loss: 0.1190 - accuracy: 0.9613
Epoch 4/5
7/7 [==============================] - 2s 269ms/step - loss: 0.0816 - accuracy: 0.9800
Epoch 5/5
7/7 [==============================] - 2s 268ms/step - loss: 0.0631 - accuracy: 0.9862


In [ ]:
loss, acc = model.evaluate(x_test, y_test)
print(f"測試資料的 loss 為: {loss:.4f}")
print(f"測試資料的正確率為: {acc*100:.2f}%")

7/7 [==============================] - 2s 187ms/step - loss: 0.0604 - accuracy: 0.9850
測試資料的 loss 為: 0.0604
測試資料的正確率為: 98.50%


In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 

In [ ]:
import gradio as gr

In [ ]:
labels = ['貓', '狗']

In [ ]:
def classify_image(inp):
  inp = inp.reshape((-1, 224, 224, 3))
  inp = preprocess_input(inp)
  p = model.predict(inp).flatten()[0]
  return {'貓': float(1-p), '狗': float(p)}

In [ ]:
image = gr.inputs.Image(shape=(224, 224), label="狗或貓的照片")
label = gr.outputs.Label(label="AI辨識結果")

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:259: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


In [ ]:
gr.Interface(fn=classify_image, inputs=image, outputs=label,
             title="AI 狗貓辨識機",
             description="請輸入一張狗或貓的照片, 看我是否分得出來!"
             ).launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>